In [4]:
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
import urllib
import os
import pandas as pd
from selenium import webdriver
from time import sleep
import re
from selenium.webdriver.common.keys import Keys

In [9]:
def page_scrap (job_url):
    
    driver = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
    driver.get(job_url)
    sleep(5)
    html = driver.page_source
    sleep(3)
    driver.close()
    soup = BeautifulSoup(html, 'lxml', from_encoding="utf-8")
    
    job_title = []
    company = []
    job_id = []
    seniority = []
    job_cat = []
    salary = []
    requirements = []
    post_date = []

    for entry in soup.find_all('h1', {'id': "job_title"}):
        job_title.append(entry.renderContents())

    for entry in soup.find_all('p', {'name': "company"}):
        company.append(entry.renderContents())

    for entry in soup.find_all('span', {'class': "black-60 db f6 fw4 mv1"}):
        job_id.append(entry.renderContents())
    
    for entry in soup.find_all('p', {'id': "seniority"}):
        seniority.append(entry.renderContents())

    for entry in soup.find_all('p', {'id': "job-categories"}):
        job_cat.append(entry.renderContents()) 

    for entry in soup.find_all('span', {'id': "last_posted_date"}):
        post_date.append(entry.renderContents())     

    salary_all = soup.find_all("span", attrs={'class': 'dib'})
    list_of_inner_text = [x.text for x in salary_all]
    salary_text = '/ '.join(list_of_inner_text)
    salary.append(salary_text)

    requirement_all = soup.find_all("div", attrs={'id': 'requirements'})
    list_of_inner_text = [x.text for x in requirement_all]
    requirements_text = ', '.join(list_of_inner_text)
    requirements.append(requirements_text)

    data = pd.DataFrame(columns = ['Job_Id','Job_Title','Company','Date_Posted','Salary',
                                   'Seniority','Category','Requirements'])

    data = data.append({'Job_Id':job_id,'Job_Title':job_title,'Company':company,
                    'Date_Posted':post_date,'Salary':salary,'Seniority':seniority,
                    'Category':job_cat,'Requirements':requirements},ignore_index=True) 

    return data

In [17]:
driver = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
driver.get("https://www.mycareersfuture.sg/")
elem = driver.find_element_by_name("search-text")
elem.send_keys("data science") #job search keyword
elem.send_keys(Keys.RETURN)
sleep(3)
current_url = driver.current_url
sleep(3)
driver.close()

y=0
links = []

for page in range(3):
    
    new_next_url = current_url.replace('page=' + str(y),'page=' +str(page))
    #print(new_next_url)

    driver2 = webdriver.Chrome(executable_path="/Users/yesplum/desktop/GApro/chromedriver/chromedriver")
    driver2.get(new_next_url)
    sleep(3)
    html1 = driver2.page_source 
    driver2.close()

    soup = BeautifulSoup(html1, 'lxml', from_encoding="utf-8")

    for entry in soup.find_all("a", attrs={'class': re.compile(r'JobCard')}):
        link = entry.get('href')
        links.append(link)

/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [18]:
len(links)

60

In [19]:
job_df = pd.DataFrame()

for i in range (len(links)):
    job_url  =  "https://www.mycareersfuture.sg%s" % links[i]
    temp_df = page_scrap(job_url)
    job_df = job_df.append(temp_df, ignore_index=True)

In [20]:
job_df.shape

(60, 8)

In [21]:
job_df.to_csv('ds.csv')